In [1]:
import os
import json
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
def generate_projects_json(overwrite=False):
  """
  Scans subfolders in the current directory for index.html files and
  generates project data JSON:
      - slug: folder name
      - title: from meta og:title
      - description: from meta og:description
      - meta: dictionary of all meta tags with name="project:<property>"
      - created: ISO 8601 formatted creation time of index.html

  The array is sorted by creation time before output.

  If overwrite=True, writes to 'projects_data.json'. Otherwise, prints to console.
  """
  output_file = "projects_data.json"
  projects = []

  root = os.getcwd()
  for folder_name in os.listdir(root):
    folder_path = os.path.join(root, folder_name)
    index_path = os.path.join(folder_path, "index.html")

    if not os.path.isdir(folder_path) or not os.path.isfile(index_path):
      continue

    with open(index_path, "r", encoding="utf-8") as f:
      soup = BeautifulSoup(f, "html.parser")

    title_meta = soup.find("meta", property="og:title")
    desc_meta = soup.find("meta", property="og:description")

    if not title_meta or not desc_meta:
      print(f"Skipping {folder_name}: missing og:title or og:description")
      continue

    meta_tags = {}
    for meta in soup.find_all("meta", attrs={"name": True}):
      name_attr = meta["name"].strip()
      if name_attr.startswith("project:"):
        key = name_attr[len("project:"):].strip()
        meta_tags[key] = meta.get("content", "").strip()

    created_iso = datetime.fromtimestamp(os.path.getctime(index_path)).isoformat()

    project_data = {
        "title": title_meta.get("content", "").strip(),
        "description": desc_meta.get("content", "").strip(),
        "slug": folder_name,
        "meta": meta_tags,
        "created": created_iso
    }

    projects.append(project_data)

  # Sort by creation time, newest first
  projects.sort(key=lambda x: x["created"], reverse=True)

  if overwrite:
    with open(output_file, "w", encoding="utf-8") as f:
      json.dump(projects, f, indent=2)
    print(f"✅ Generated {output_file} with {len(projects)} projects.")
  else:
    print(json.dumps(projects, indent=2))

In [3]:
generate_projects_json(overwrite=True)

✅ Generated projects_data.json with 11 projects.
